# Socrata API

For this project you will use `requests` package and the SODA API to access data through https://data.nashville.gov/. We'll start out with something familiar, the [Top 500 Monthly Searches](https://data.nashville.gov/Public-Services/Nashville-gov-Top-500-Monthly-Searches/fuaa-r5cm), then pull in different datasets further on. You will make different API requests for each individual question.

Each dataset has its own api endpoint. You can find the endpoint for a dataset by clicking on the `API` button in the top right of the dataset screen, then copying the `API Endpoint`. The default output is `JSON`, which you can leave unchanged:

![api_endpoint](../assets/api_endpoint.png)

Each API is different, so it is very important to read the documentation for each API to know how to use it properly. The documentation for the SODA API is [here](https://dev.socrata.com/consumers/getting-started.html). It is **HIGHLY RECOMMENDED** that you read the documentation before making any requests, then do deeper dives into specific use cases when questions require. NOTE that the examples in the documentation don't use the `requests` package. You will need to look at the examples and figure out which things go in the `url` and which things go in the `params`.


In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#with open('../../../../../tech/api_keys.json') as file: credentials = json.load(file)

In [ ]:
#credentials.keys()

In [ ]:
#api_key = credentials['api key']

### Questions  


**1. Make an API request that returns the months where "fire" was searched in 2016. Which month had the most searches?**

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json'

In [ ]:
params = {
    'year': 2016,
    'query_text': 'fire'
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res = response.json()

In [ ]:
fire_2016_queries_df = pd.DataFrame(res)

In [ ]:
most_searches = fire_2016_queries_df.sort_values(by='query_count', ascending=False).iloc[0]

print(most_searches.month_name, ' had ', most_searches.query_count, ' searches for fire in 2016, the most of any month.')

**2. Make an API request that returns all the times a query was run more than 100 times in a month. How many times did this occur?**  

In [ ]:
params = {
    '$where': 'query_count > 100',
    '$limit': 5000
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res = response.json()

In [ ]:
print(len(res),' queries have been run more than 100 times in a month.')

**3. Make another API request that returns all the times "codes" was searched more than 100 times in a month. How many times did this occur?** 

In [ ]:
params = {
    '$where': 'query_count > 100',
    'query_text': 'codes',
    '$limit': 5000
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res = response.json()

In [ ]:
print('There have been ', len(res), ' months in which codes was searched > 100 times.')

**4. Make an API request that returns the entire Top 500 Monthly Searches dataset. Make a chart that shows the number of times "maps" was searched in a month across the entire time frame.**

In [ ]:
params = {
    '$limit': 50000
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res = response.json()

In [ ]:
print(len(res),' total rows in Top 500 Monthly Searches dataset')

In [ ]:
complete_dataset_df = pd.DataFrame(res)
complete_dataset_df.head()

### MAPS subset

In [ ]:
maps_queries_df = complete_dataset_df.loc[complete_dataset_df.query_text.str.upper() == 'MAPS']

In [ ]:
print(maps_queries_df.query_text.value_counts())
maps_queries_df.head()

### month_year_date

In [ ]:
month_name_series = maps_queries_df['month_name'] + ' ' + maps_queries_df['year']

In [ ]:
maps_queries_df['month_year_date'] = pd.to_datetime(month_name_series)
maps_queries_df.head()

### query_count to int

In [ ]:
maps_queries_df = maps_queries_df.astype({'query_count':int})
maps_queries_df.dtypes

### Columns Subset

In [ ]:
maps_queries_df = maps_queries_df.loc[:,['month_year_date', 'query_count']]
maps_queries_df.head()

### Group by 

In [ ]:
maps_queries_df = maps_queries_df.groupby(['month_year_date']).agg('sum')
maps_queries_df = maps_queries_df.sort_values(by='month_year_date')
maps_queries_df = maps_queries_df.reset_index()
maps_queries_df.head()

In [ ]:
maps_queries_df.plot('month_year_date',
                     'query_count',
                     figsize=(12,6),
                     legend=None)

plt.title('"Maps" Queries per Month', fontsize=20)
plt.ylabel('Number of Queries', fontsize=16)
plt.xlabel('')
plt.xticks(rotation='horizontal', fontsize =14, ha='center')
plt.yticks(fontsize=14)
#plt.grid()
;

### Stretch Questions

**5. Make an API request to pull back all the data from [hubNashville (311) Service Requests](https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh) (check to see how many rows you can return in a single request). Compare it to the Top 500 Monthly Searches data set. What do you observe? (This is open-ended, there isn't a specific answer for this one)**

In [ ]:
endpoint = 'https://data.nashville.gov/resource/7qhx-rexh.json'

In [ ]:
params = {
#    '$limit': 2000000
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res_hub_nash = response.json()

In [ ]:
len(res_hub_nash)

In [ ]:
print(len(res_hub_nash), ' rows are in the hubNashville (311) Service Requests dataset. ')
print('All of them can be returned in a single request')

**6. Find 2 new data sets on data.nashville.gov, make API requests to pull the data, and do an analysis that combines the data sets.**

## NES - Monthly Energy Consumption by Customer Type and ZIP Code

https://data.nashville.gov/Energy-Usage/NES-Monthly-Energy-Consumption-by-Customer-Type-an/vbx7-mn5i

In [ ]:
endpoint = 'https://data.nashville.gov/resource/vbx7-mn5i.json'

In [ ]:
params = {
    '$limit': 20000
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res = response.json()

In [ ]:
energy_cons_df = pd.DataFrame(res)
energy_cons_df.head()

In [ ]:
#Keep only rows with year <= 2019. The data is limited/unreliable after that.
energy_cons_df = energy_cons_df.loc[energy_cons_df.year.notna()]
#energy_cons_df = energy_cons_df.loc[energy_cons_df.year.astype(int) <= 2019]

In [ ]:
energy_cons_df['monthly_kwh_used'] = energy_cons_df['monthly_kwh_used'].astype(int)
energy_cons_df['year'] = energy_cons_df['year'].astype(int)
energy_cons_df['month'] = energy_cons_df['month'].astype(int)
energy_cons_df.dtypes

In [ ]:
#Add month_year_date column
date_columns = energy_cons_df.loc[:,['year','month']]
date_columns['day'] = 1
energy_cons_df['month_year_date']=pd.to_datetime(date_columns)
energy_cons_df

### Annual Graph 1

In [ ]:
#Subset of columns for graph
energy_cons_cust_df = energy_cons_df.copy()
energy_cons_cust_df = energy_cons_cust_df.loc[:,['year','customer_type','monthly_kwh_used']]

In [ ]:
energy_cons_cust_df = energy_cons_cust_df.loc[energy_cons_cust_df.year.astype(int) <= 2019]

In [ ]:
energy_cons_cust_df = energy_cons_cust_df.groupby(['year','customer_type']).agg('sum')
energy_cons_cust_df = energy_cons_cust_df.sort_values(by=['year','customer_type'])
energy_cons_cust_df = energy_cons_cust_df.reset_index()
energy_cons_cust_df.head()

In [ ]:
energy_cons_cust_df['annual_gwh_used'] = energy_cons_cust_df['monthly_kwh_used'] * 0.001 * 0.001
energy_cons_cust_df.head(2)

In [ ]:
hue_order=['COMCOMM > 50 KW',
           'RESRESIDENTIAL                SRSRESID. SUPPL.',
           'COMCOMM =< 50 KW              COMCOMM 3-PHASE',
           'SEASEAS > 50 KW',
           'SEASEAS =< 50 KW',
           'OLMOLB METERED',
           'COMMERCIAL',
           'RESIDENTIAL',
           'OUTDOOR LIGHTING',
           'SEASONAL CUSTOMER']

p = sns.lineplot(x='year',
             y='annual_gwh_used',
             data=energy_cons_cust_df,
             hue='customer_type',
             hue_order = hue_order,
             ci=None)
plt.legend(loc='upper right',bbox_to_anchor=(1.85,1))

p.set_title('NES: Annual Energy Consumption by Customer Type', fontsize=16)
p.set_ylabel('Annual GWh Used', fontsize=14)
p.set_xlabel('')
#plt.xticks(rotation='horizontal', fontsize =14, ha='center')
#plt.yticks(fontsize=14)
#plt.grid()
;

Data is missing for three months in 2018: March, September, October

### Correct Customer Types

In [ ]:
cust_type_changes = {'COMCOMM =< 50 KW              COMCOMM 3-PHASE':'Commercial',
                     'COMCOMM > 50 KW':'Commercial',
                     'OLMOLB METERED':'Metered',
                     'RESRESIDENTIAL                SRSRESID. SUPPL.':'Residential',
                     'SEASEAS =< 50 KW':'Seasonal Customer',
                     'SEASEAS > 50 KW':'Seasonal Customer',
                     'COMMERCIAL':'Commercial',
                     'OUTDOOR LIGHTING':'Outdoor Lighting',
                     'RESIDENTIAL':'Residential',
                     'SEASONAL CUSTOMER':'Seasonal Customer'
                    }

In [ ]:
energy_cons_df.replace(to_replace=cust_type_changes, inplace=True)

### Annual Graph 2

In [ ]:
#Subset of columns for graph
energy_cons_cust_df2 = energy_cons_df.copy()
energy_cons_cust_df2 = energy_cons_cust_df2.loc[:,['year','customer_type','monthly_kwh_used']]

In [ ]:
energy_cons_cust_df2 = energy_cons_cust_df2.groupby(['year','customer_type']).agg('sum')
energy_cons_cust_df2 = energy_cons_cust_df2.sort_values(by=['year','customer_type'])
energy_cons_cust_df2 = energy_cons_cust_df2.reset_index()
energy_cons_cust_df2.head()

In [ ]:
energy_cons_cust_df2 = energy_cons_cust_df2.loc[energy_cons_cust_df2.year.astype(int) <= 2019]

In [ ]:
energy_cons_cust_df2['annual_gwh_used'] = energy_cons_cust_df2['monthly_kwh_used'] * 0.001 * 0.001
energy_cons_cust_df2

In [ ]:
hue_order=['Commercial',
           'Residential',
           'Seasonal Customer',
           'Metered',
           'Outdoor Lighting']
fig, ax = plt.subplots(figsize=(10,5))
p = sns.lineplot(x='year',
                 y='annual_gwh_used',
                 data=energy_cons_cust_df2,
                 hue='customer_type',
                 hue_order = hue_order,
                 ax=ax,
                 marker='o',
                 markersize=4,
                 ci=None)
plt.legend(loc='upper left',bbox_to_anchor=(1.02,1))
plt.text(x = 2018, y=3000, s='2018 has 3 months missing', fontsize = 10,ha='center')
p.set_title('NES: Annual Energy Consumption by Customer Type', fontsize=16)
p.set_ylabel('Annual GWh Used', fontsize=14)
p.set_xlabel('')
;

### Decrease in annual energy consumption from 2012 to 2019
#### - Commercial: 12% decrease
#### - Residential: 27% decrease

### Monthly Energy Consumption

In [ ]:
#years = mdates.YearLocator()  #every year
#month = mdates.MonthLocator() #every month
#years_fmt = mdates.DateFormatter('%Y-%m')

In [ ]:
#Subset of columns for graph
energy_cons_cust_df3 = energy_cons_df.copy()
energy_cons_cust_df3 = energy_cons_cust_df3.loc[:,['year','month','month_year_date','customer_type','monthly_kwh_used']]
energy_cons_cust_df3

In [ ]:
energy_cons_cust_df3 = energy_cons_cust_df3.groupby(['year','month','month_year_date','customer_type']).agg('sum')
energy_cons_cust_df3 = energy_cons_cust_df3.sort_values(by=['month_year_date','customer_type'])
energy_cons_cust_df3 = energy_cons_cust_df3.reset_index()
energy_cons_cust_df3.head()

In [ ]:
energy_cons_cust_df3['monthly_gwh_used'] = energy_cons_cust_df3['monthly_kwh_used'] * 0.001 * 0.001
energy_cons_cust_df3.head()

In [ ]:
#Remove all customer types except Commercial and Residential
energy_cons_cust_df3 = energy_cons_cust_df3.loc[energy_cons_cust_df3.customer_type.isin(['Commercial','Residential'])]

In [ ]:
#Remove two outlier points
index_names = energy_cons_cust_df3.loc[(energy_cons_cust_df3.customer_type == "Commercial") 
                         & (energy_cons_cust_df3.monthly_gwh_used < 300)].index
index_names

In [ ]:
energy_cons_cust_df3.drop(index_names, inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
p = sns.lineplot(x='month_year_date',
                 y='monthly_gwh_used',
                 data=energy_cons_cust_df3,
                 ax=ax,
                 hue='customer_type',
                 ci=None,
                 marker='o',
                 markersize=4)
plt.legend(loc='upper right',bbox_to_anchor=(1,1))

p.set_title('NES - Monthly Energy Consumption by Customer Type', fontsize=16)
p.set_ylabel('Monthly GWh Used', fontsize=14)
p.set_xlabel('')
p.set_xticks([pd.to_datetime('01/01/2012'), pd.to_datetime('01/01/2013'),
              pd.to_datetime('01/01/2014'), pd.to_datetime('01/01/2015'),
              pd.to_datetime('01/01/2016'), pd.to_datetime('01/01/2017'),
              pd.to_datetime('01/01/2018'), pd.to_datetime('01/01/2019'),
              pd.to_datetime('01/01/2020'), pd.to_datetime('01/01/2021'),
              pd.to_datetime('01/01/2022')])
p.set_xticklabels(['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'])
p.set_ylim(bottom=0,top=600)
;

## NES - Monthly Energy Consumption Greater than 50 Kilowatts

In [ ]:
endpoint = 'https://data.nashville.gov/resource/ut4c-a9ss.json'

In [ ]:
params = {
#    'year': 2016,
#    'query_text': 'fire'
}

In [ ]:
response = requests.get(endpoint, params = params)

if response.status_code != 200:
    print(response.status_code)
    assert response.status_code == 200
    
res50kw = response.json()

In [ ]:
e50kw_df = pd.DataFrame(res50kw)
print(e50kw_df.shape)
e50kw_df.head()

In [ ]:
e50kw_df.reportyear.max()

In [ ]:
e50kw_df.drop_duplicates(inplace=True)

In [ ]:
#tvacode, kwd, and kwh all have nulls
e50kw_df = e50kw_df.loc[e50kw_df.kwh.notna()]

In [ ]:
e50kw_df['year'] = e50kw_df['reportyear'].astype(int)
e50kw_df['month'] = e50kw_df['reportmonth'].astype(int)
e50kw_df['nbrofcustomers'] = e50kw_df['nbrofcustomers'].astype(int)
e50kw_df['kwh'] = e50kw_df['kwh'].astype(int)
e50kw_df.dtypes

In [ ]:
print(e50kw_df.shape)
e50kw_df = e50kw_df.loc[e50kw_df.kwh != 0]
print(e50kw_df.shape)

In [ ]:
index_row = e50kw_df.loc[(e50kw_df.demand == '1001-5000 KW, ANNUAL') & (e50kw_df.kwh < 30000000) & (e50kw_df.year == 2012)].index
e50kw_df.drop(index_row, inplace = True)

In [ ]:
#Add month_year_date column
date_columns = e50kw_df.loc[:,['year','month']]
date_columns['day'] = 1
e50kw_df['month_year_date']=pd.to_datetime(date_columns)
e50kw_df

In [ ]:
#Subset of columns for graph
e50kw_df = e50kw_df.loc[:,['month_year_date','demand','nbrofcustomers','kwh']]

In [ ]:
#Group by date and demand type with aggregations for kwh and nbrofcustomers.
agg_dict = {'nbrofcustomers':'mean','kwh':'sum'}

e50kw_df = e50kw_df.groupby(['month_year_date','demand']).agg(agg_dict)
e50kw_df = e50kw_df.sort_values(by=['month_year_date','demand'])
e50kw_df = e50kw_df.reset_index()
e50kw_df.head()

In [ ]:
#Add column with average KWh used per customer
e50kw_df['kwh_per_cust'] = e50kw_df['kwh'] / e50kw_df['nbrofcustomers']

In [ ]:
#Add columns with units in GWh rather than KWh
e50kw_df['gwh'] = e50kw_df['kwh'] * 0.001 * 0.001
e50kw_df['gwh_per_cust'] = e50kw_df['kwh_per_cust'] * 0.001 * 0.001
e50kw_df.tail(6)

In [ ]:
hue_order = ['Total',
             '51-1000 KW, ANNUAL',
             '1001-5000 KW, ANNUAL',
             'LIP',
             'OVER 50000',
             '51-1000 KW, SEASONAL']
fig, ax = plt.subplots(figsize=(10,5))
p = sns.lineplot(x='month_year_date',
                 y='gwh',
                 data=e50kw_df,
                 ax=ax,
                 hue='demand',
                 hue_order=hue_order,
                 ci=None,
                 marker='o',
                 markersize=4)
plt.legend(loc='upper right',bbox_to_anchor=(0.91,1))

p.set_title('NES: Consumption by Commercial Customers Using > 50 KWh Monthly', fontsize=16)
p.set_ylabel('Monthly GWh Used', fontsize=14)
p.set_xlabel('')
p.set_xticks([pd.to_datetime('01/01/2011'),
              pd.to_datetime('01/01/2012'), pd.to_datetime('01/01/2013'),
              pd.to_datetime('01/01/2014'), pd.to_datetime('01/01/2015'),
              pd.to_datetime('01/01/2016'), pd.to_datetime('01/01/2017'),
              pd.to_datetime('01/01/2018'), pd.to_datetime('01/01/2019'),
              pd.to_datetime('01/01/2020'), pd.to_datetime('01/01/2021'),
              pd.to_datetime('01/01/2022')])
p.set_xticklabels(['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'])
p.set_ylim(bottom=0,top=500)
;

In [ ]:
#Multiply kwh (and gwh) values times 1000 for records between July 2015 and Jan 2020.
#Reason: kwh values appear to be 1000 times smaller for records between July 2015 and Jan 2020, which means they 
#were probably recorded in MWh rather than KWh. 

index_rows = e50kw_df.loc[(e50kw_df.month_year_date >= pd.to_datetime('07/01/2015')) 
                          & (e50kw_df.month_year_date <=  pd.to_datetime('01/01/2020'))].index

e50kw_df.loc[e50kw_df.index.isin(index_rows),'kwh'] = e50kw_df.loc[e50kw_df.index.isin(index_rows),'kwh'] * 1000
e50kw_df.loc[e50kw_df.index.isin(index_rows),'gwh'] = e50kw_df.loc[e50kw_df.index.isin(index_rows),'gwh'] * 1000
e50kw_df.loc[e50kw_df.index.isin(index_rows),'kwh_per_cust'] = e50kw_df.loc[e50kw_df.index.isin(index_rows),'kwh_per_cust'] * 1000
e50kw_df.loc[e50kw_df.index.isin(index_rows),'gwh_per_cust'] = e50kw_df.loc[e50kw_df.index.isin(index_rows),'gwh_per_cust'] * 1000

In [ ]:
#Remove outlier rows
#index_row_1 = e50kw_df.loc[(e50kw_df.demand == '51-1000 KW, ANNUAL') & (e50kw_df.gwh < 200)].index
#e50kw_df.drop(index_row_1, inplace = True)

#index_row_2 = e50kw_df.loc[(e50kw_df.demand == '1001-5000 KW, ANNUAL') & (e50kw_df.gwh < 30)].index
#e50kw_df.drop(index_row_2, inplace = True)

In [ ]:
hue_order = ['Total',
             '51-1000 KW, ANNUAL',
             '1001-5000 KW, ANNUAL',
             'LIP',
             'OVER 50000',
             '51-1000 KW, SEASONAL']
fig, ax = plt.subplots(figsize=(12,6.5))
p = sns.lineplot(x='month_year_date',
                 y='gwh',
                 data=e50kw_df,
                 ax=ax,
                 hue='demand',
                 hue_order=hue_order,
                 ci=None,
                 marker='o',
                 markersize=4)
plt.legend(loc='upper right',bbox_to_anchor=(1,1))

p.set_title('NES: Consumption by Commercial Customers Using > 50 KWh Monthly', fontsize=16)
p.set_ylabel('Monthly GWh Used', fontsize=14)
p.set_xlabel('')
p.set_xticks([pd.to_datetime('01/01/2011'),
              pd.to_datetime('01/01/2012'), pd.to_datetime('01/01/2013'),
              pd.to_datetime('01/01/2014'), pd.to_datetime('01/01/2015'),
              pd.to_datetime('01/01/2016'), pd.to_datetime('01/01/2017'),
              pd.to_datetime('01/01/2018'), pd.to_datetime('01/01/2019'),
              pd.to_datetime('01/01/2020'), pd.to_datetime('01/01/2021'),
              pd.to_datetime('01/01/2022')])
p.set_xticklabels(['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'])
p.set_ylim(bottom=0,top=500)
;

In [ ]:
hue_order = ['51-1000 KW, ANNUAL',
             '1001-5000 KW, ANNUAL',
             'LIP',
             'OVER 50000',
             '51-1000 KW, SEASONAL']
fig, ax = plt.subplots(figsize=(12,6.5))
p = sns.lineplot(x='month_year_date',
                 y='gwh_per_cust',
                 data=e50kw_df,
                 ax=ax,
                 hue='demand',
                 hue_order=hue_order,
                 ci=None,
                 marker='o',
                 markersize=4)
plt.legend(loc='upper right',bbox_to_anchor=(1,1))

p.set_title('NES: Average Consumption per Commercial Customer Using > 50 KWh Monthly', fontsize=16)
p.set_ylabel('Monthly GWh Used per Customer', fontsize=14)
p.set_xlabel('')
p.set_xticks([pd.to_datetime('01/01/2011'),
              pd.to_datetime('01/01/2012'), pd.to_datetime('01/01/2013'),
              pd.to_datetime('01/01/2014'), pd.to_datetime('01/01/2015'),
              pd.to_datetime('01/01/2016'), pd.to_datetime('01/01/2017'),
              pd.to_datetime('01/01/2018'), pd.to_datetime('01/01/2019'),
              pd.to_datetime('01/01/2020'), pd.to_datetime('01/01/2021'),
              pd.to_datetime('01/01/2022')])
p.set_xticklabels(['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'])
#p.set_ylim(bottom=0,top=500)
;

In [ ]:
e50kw_per_cust_df = e50kw_df.copy()
e50kw_per_cust_df = e50kw_per_cust_df.loc[e50kw_per_cust_df.demand != 'Total',['demand','nbrofcustomers','gwh_per_cust','gwh']]

In [ ]:
#Group by date and demand type to get average number of customers, average kwh per customer, and average gwh per customer.
e50kw_per_cust_df = e50kw_per_cust_df.groupby(['demand']).agg('mean')
e50kw_per_cust_df = e50kw_per_cust_df.sort_values(by=['gwh'],ascending=False)
e50kw_per_cust_df = e50kw_per_cust_df.reset_index()
e50kw_per_cust_df

In [ ]:
hue_order = ['51-1000 KW, ANNUAL',
             '1001-5000 KW, ANNUAL',
             'LIP',
             'OVER 50000',
             '51-1000 KW, SEASONAL']
fig, ax = plt.subplots(figsize=(5,5))
p = sns.scatterplot(x='nbrofcustomers',
                 y='gwh_per_cust',
                 data=e50kw_per_cust_df,
                 ax=ax,
                 hue='demand',
                 hue_order=hue_order,
                 ci=None)
plt.legend(loc='upper right',bbox_to_anchor=(1,1))

p.set_title('NES: Average Consumption per Commercial Customer Using > 50 KWh Monthly', fontsize=12)
p.set_ylabel('Monthly GWh Used per Customer', fontsize=10)
p.set_xlabel('Number of Customers', fontsize=10)
#p.set_xticks([pd.to_datetime('01/01/2011'),
#              pd.to_datetime('01/01/2012'), pd.to_datetime('01/01/2013'),
#              pd.to_datetime('01/01/2014'), pd.to_datetime('01/01/2015'),
#              pd.to_datetime('01/01/2016'), pd.to_datetime('01/01/2017'),
#              pd.to_datetime('01/01/2018'), pd.to_datetime('01/01/2019'),
#              pd.to_datetime('01/01/2020'), pd.to_datetime('01/01/2021'),
#              pd.to_datetime('01/01/2022')])
#p.set_xticklabels(['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'])
#p.set_ylim(bottom=0,top=500)
;

#### Bonus

7. Socrata is used by many cities, states, and federal organizations. Find additional datasets through [Socrata's Open Data Network](http://www.opendatanetwork.com/) and do an analysis comparing them to Nashville or each other.

#### Show and Tell
At the end of the project you will present some general insights, visualizations, or other finding from any part of the project. This will be informal (showing your Jupyter notebook is fine, no need to make a powerpoint) and should be no more than 5 min. If you had challenges making your visualizations, then it is fine to discuss your experience working with the API and what you were intending to show.